# collect ICD9/10 codes

In [ ]:
#import required packages
import tqdm
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd

import matplotlib as mpl
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display, HTML
from datetime import date
import multiprocessing

import os
import statsmodels.api as sm

import pickle

In [ ]:
#call the workspace bucket and CDR directory; this command will be repeated in most analyses in this repository
my_bucket = os.getenv('WORKSPACE_BUCKET')
my_bucket
CDR_version=os.getenv("WORKSPACE_CDR")

In [ ]:
#important files for phecode annotation
sex_at_birth_restriction=pd.read_csv(f'{my_bucket}/data/phewas/sex_at_birth_restriction.csv')
sex_at_birth_restriction.head(5)

phecodes=pd.read_csv(f'{my_bucket}/data/phewas/ICDPhecodes')
phecodes.head(500)

In [ ]:
import os

# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_04499009_person_sql = """
    SELECT
        person.person_id 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person   
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (836793) 
                                AND is_standard = 0  
                                AND  value_source_concept_id IN (1384519)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (194992) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) )"""

ced_person = pd.read_gbq(
    dataset_04499009_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

ced_person.head(5)

In [ ]:
ced_person['celiac']=1

In [ ]:
# now find everyone without CeD
# amend to include people with icd9cm and icd10 cm in observation
# use df_smoke_controls_exclusion person_ids since we don't want anyone with chewing tobacco in controls
query = ("""
select distinct person_id from `"""+ str(CDR_version)+""".condition_occurrence`
where person_id not in """+"("+' '.join([str(id)+"," for id in np.unique(ced_person.person_id)])[:-1]+")"+""";
""")
no_cel = pd.read_gbq(query, dialect="standard")
# people with at least one icd9/10cm code in observation but no smoking code (including chewing tobacco)
# so use df_smoke_controls_exclusion person_ids
# deduplicate the person_ids
query = ("""
SELECT distinct person_id
FROM 
    (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date
        FROM `"""+ str(CDR_version) +""".observation`) AS obs
     INNER JOIN 
        (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id 
            FROM `"""+str(CDR_version)+""".concept`
            where (vocabulary_id ='ICD9CM') or 
            (vocabulary_id ='ICD10CM')) as concept 
            on concept.concept_id = obs.observation_source_concept_id
       where person_id not in """+"("+' '.join([str(id)+"," for id in np.unique(ced_person.person_id)])[:-1]+")"+"""
""")
observation_coded_participants_no_cel = pd.read_gbq(query, dialect="standard")

# append to cel_nos from condition occurrence
no_cel = pd.concat([no_cel,observation_coded_participants_no_cel])
# deduplicate 
no_cel = no_cel.drop_duplicates()
no_cel["celiac"] = 0

ced_ehr = pd.concat([ced_person, no_cel])

In [ ]:
ced_ehr.value_counts('celiac')
#254777 healhty, 3040 CeD

## query for covariates

In [ ]:
from datetime import date

def age(birthdate,enrolldate):
    age = enrolldate.year - birthdate.year - ((enrolldate.month, enrolldate.day) < (birthdate.month, birthdate.day))
    return age


In [ ]:
def make_covariates(df_indep_var,CDR_version):
    # first query from condition occurrence to get number of distinct codes and ehr length
    query="""SELECT person_id, min(condition_start_date) as min_cond_date,max(condition_start_date)as max_cond_date,COUNT(DISTINCT condition_concept_id) as cond_code_cnt 
    FROM `"""+CDR_version+""".condition_occurrence` WHERE condition_start_date >='1980-01-01'
    GROUP BY person_id """

    ehr_covariate=pd.read_gbq(query, dialect="standard")
    # EHR Length is a bit subtle here. Observation date is a little tricky because birthdate is used in observation. 
    # What we need to do is take the relevant codes from observation and find the minimum and maximum dates overall
    query = ("""
        SELECT distinct person_id, min(observation_date) as min_obs_date, max(observation_date) as max_obs_date, COUNT(DISTINCT observation_concept_id) as observation_code_cnt 
        FROM 
            (SELECT DISTINCT person_id, observation_source_concept_id, observation_source_value, observation_date, observation_concept_id
                FROM `"""+ str(CDR_version) +""".observation`) AS obs
             INNER JOIN 
                (SELECT DISTINCT concept_id, concept_name, concept_code, vocabulary_id 
                    FROM `"""+str(CDR_version)+""".concept`
                    where (vocabulary_id ='ICD9CM') or 
                    (vocabulary_id ='ICD10CM')) as concept 
                    on concept.concept_id = obs.observation_source_concept_id
        group by person_id
        """)
    observation_code_dates = pd.read_gbq(query, dialect="standard")
    # note, this procedure takes into consideration NaTs, so if a person didn't have an observation code, it will
    ehr_covariate_plus = pd.merge(observation_code_dates, ehr_covariate, on="person_id", how = 'outer')
    # automatically take the minimum condition occurrence date.
    ehr_covariate_plus["min_date"] = [ehr_covariate_plus[["min_obs_date","min_cond_date"]].loc[i].min() for i in ehr_covariate_plus.index]
    ehr_covariate_plus["max_date"] = [ehr_covariate_plus[["max_obs_date","max_cond_date"]].loc[i].max() for i in ehr_covariate_plus.index]
    ehr_covariate_plus["code_cnt"] =  ehr_covariate_plus[["cond_code_cnt","observation_code_cnt"]].sum(axis =1)
    # for efficiency sake drop the extra columns
    ehr_covariate_plus = ehr_covariate_plus.drop(columns = ["min_obs_date","min_cond_date","max_obs_date","max_cond_date","cond_code_cnt","observation_code_cnt"])
    # ehr length defined as time between minimum of relevant observation dates  
    ehr_covariate_plus["ehr_length"]=(ehr_covariate_plus['max_date']-ehr_covariate_plus['min_date']).apply(lambda x:x).dt.days

    # Now Demographics
    
    query="""SELECT DISTINCT p.person_id,sex_at_birth_concept_id,gender_concept_id,race_concept_id,ethnicity_concept_id,birth_datetime FROM 
    `"""+CDR_version+""".person` p 
    """
    demo_patients=pd.read_gbq(query, dialect="standard")
    print(demo_patients)
    demo_patients["age_today"] = (datetime.now(tz=timezone.utc)-demo_patients["birth_datetime"])/np.timedelta64(1,'Y')
    demo_patients=demo_patients[demo_patients["person_id"].isin(df_indep_var["person_id"])]
    
    # tidy up 
    #demo_patients_merge = demo_patients
    #demo_patients_merge=pd.merge(demo_patients, race_generalized,on="person_id")
    demo_patients_merge=pd.merge(demo_patients, ehr_covariate_plus, on="person_id") # now merge to other ehr covariates # amended 04/17/2020
    demo_patients_merge["age_at_last_event"] = (pd.to_datetime(demo_patients_merge['max_date'],utc=True)-demo_patients_merge["birth_datetime"])/np.timedelta64(1,'Y')
    
    #############
    #race
    #############
    white = 8527
    black_aa = 8516
    asian = 8515
    # other 
    more_than_one = 2000000008
    none_of_these = 45882607
    another_single_pop=2000000001
    other = [more_than_one, none_of_these,another_single_pop]
    #Unknown
    prefer_not_answer = 1177221
    skip = 903096
    no_match = 0
    unknown_race = [prefer_not_answer, skip, no_match]
    ## update indicators
    demo_patients_merge["white"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["AF"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["Asian"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["race_unk"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["Other"]=[0]*demo_patients_merge.shape[0]
    
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==white,"white"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==black_aa,"AF"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"]==asian,"Asian"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"].isin(unknown_race),"race_unk"]=1
    demo_patients_merge.loc[demo_patients_merge["race_concept_id"].isin(other),"Other"]=1

    
    #############
    #ethnicity
    #############
    #hisp/latino
    hisp_latino = 38003563
    # not hisp/latino
    not_hisp_lat = 38003564
    none_of_these = 45882607
    not_hisp = [not_hisp_lat, none_of_these]
    # unknown
    prefer_no_ans = 1177221
    skip= 903096
    unknown_eth = [prefer_no_ans, skip]
    # update indicators 
    
    demo_patients_merge["hisp_lat"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["not_hisp_lat"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["unk_eth"]=[0]*demo_patients_merge.shape[0]
    # sex at birth 
    demo_patients_merge.loc[ demo_patients_merge["ethnicity_concept_id"]==hisp_latino,"hisp_lat"]=1
    demo_patients_merge.loc[ demo_patients_merge["ethnicity_concept_id"]==not_hisp_lat,"not_hisp_lat"]=1
    demo_patients_merge.loc[(demo_patients_merge["ethnicity_concept_id"].isin(unknown_eth)),"unk_eth"]=1
    
    
    #############
    #sex at birth
    #############
    female = 45878463
    male = 45880669
    unknown = 2000000009 # prefer not to answer
    zer = 0
    ## update indicators
    demo_patients_merge["female"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["male"]=[0]*demo_patients_merge.shape[0]
    demo_patients_merge["unk_sex"]=[0]*demo_patients_merge.shape[0]
    # sex at birth 
    demo_patients_merge.loc[demo_patients_merge["sex_at_birth_concept_id"]==female,"female"]=1
    demo_patients_merge.loc[demo_patients_merge["sex_at_birth_concept_id"]==male,"male"]=1
    demo_patients_merge.loc[(demo_patients_merge["sex_at_birth_concept_id"]==unknown)|
                            (demo_patients_merge["sex_at_birth_concept_id"]==zer),"unk_sex"]=1
    # now merge to the indep var of interest 
    demo_patients_cov=pd.merge(demo_patients_merge,df_indep_var, on = "person_id")
    demo_patients_cov=demo_patients_cov.drop_duplicates()
    return(demo_patients_cov)

In [ ]:
demo_patients_cov = make_covariates(ced_ehr, CDR_version = CDR_version)

In [ ]:
demo_patients_cov.to_csv(f'demo_patients_v2.csv',index=False)
demo_patients_cov

In [ ]:
demo_patients_cov = pd.read_csv(f'{my_bucket}/data/phewas/demo_patients_v2.csv')

In [ ]:
demo_patients_cov.value_counts('celiac')

## map ICD codes to phecodes

In [ ]:
# More specific icd9 and 10 querying
def getPhecodeParticipants(phecodes_list, CDR_version, num_processes):
    """
    Batching function for parallel extraction of participant Phecodes 
    ======================================================================================================
    phecodes_batch: Pandas Dataframe of Phecodes 
    return_dict: 
    phecodes_list: List of phecodes to process 
    CDR_version: String of current cdr version 
    ## need to include rollup 
    """
    size = int(np.ceil(len(phecodes_list)/num_processes))
    phecodes=ICD9_IC10_Phecodes[ICD9_IC10_Phecodes["phecode"].isin(phecodes_list)]
    
    ## ICD Codes in condition_occurrence (if a code is in observation, then it shouldn't query anything here)
    icd9_codes_cond=phecodes[phecodes['vocabulary_id']=='ICD9CM']["code"].tolist()
    #
    icd9_codes_cond_str="'"+"','".join(icd9_codes_cond)+"'"
    
    icd10_codes_cond=phecodes[phecodes['vocabulary_id']=='ICD10CM']["code"].tolist()
    icd10_codes_cond_str="'"+"','".join(icd10_codes_cond)+"'"
    
    ## ICD Codes in observation 
    
    icd9_codes_obs=phecodes[phecodes['vocabulary_id']=='ICD9CM']["code"].tolist()
    #
    icd9_codes_obs_str="'"+"','".join(icd9_codes_obs)+"'"
    
    icd10_codes_obs=phecodes[phecodes['vocabulary_id']=='ICD10CM']["code"].tolist()
    icd10_codes_obs_str="'"+"','".join(icd10_codes_obs)+"'"
    
    # there's a subtlety here that we need icd10 cm
    query="""SELECT DISTINCT icd.person_id,condition_start_date as start_date,condition_concept_id as cid,concept_code,vocabulary_id FROM `"""+CDR_version+""".concept` 
    c  INNER JOIN `"""+CDR_version+""".condition_occurrence` icd  ON icd.condition_source_concept_id=c.concept_id  
    WHERE vocabulary_id ='ICD9CM' AND  concept_code IN ("""+icd9_codes_cond_str+""") 
    ORDER BY condition_start_date"""
    icdcodes1_cond=pd.read_gbq(query, dialect="standard",use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),progress_bar_type="tqdm_notebook")
    query="""SELECT DISTINCT icd.person_id,condition_start_date as start_date,condition_concept_id as cid,concept_code, vocabulary_id FROM `"""+CDR_version+""".concept` 
    c  INNER JOIN `"""+CDR_version+""".condition_occurrence` icd  ON c.concept_id = icd.condition_source_concept_id  
    WHERE vocabulary_id ='ICD10CM' AND  concept_code IN ("""+icd10_codes_cond_str+""")
    ORDER BY condition_start_date"""
    icdcodes2_cond=pd.read_gbq(query, dialect="standard",use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),progress_bar_type="tqdm_notebook")
    
    #Now observations 
    
    query="""SELECT DISTINCT icd.person_id,observation_date as start_date, observation_concept_id as cid,concept_code,vocabulary_id FROM `"""+CDR_version+""".concept` 
    c  INNER JOIN `"""+CDR_version+""".observation` icd  ON icd.observation_source_concept_id=c.concept_id  
    WHERE vocabulary_id ='ICD9CM' AND  concept_code IN ("""+icd9_codes_obs_str+""") 
    ORDER BY start_date"""
    icdcodes1_obs=pd.read_gbq(query, dialect="standard",progress_bar_type="tqdm_notebook")
    query="""SELECT DISTINCT icd.person_id,observation_date as start_date,observation_concept_id as cid, concept_code, vocabulary_id FROM `"""+CDR_version+""".concept` 
    c  INNER JOIN `"""+CDR_version+""".observation` icd  ON c.concept_id = icd.observation_source_concept_id  
    WHERE vocabulary_id ='ICD10CM' AND  concept_code IN ("""+icd10_codes_obs_str+""")
    ORDER BY start_date"""
    
    icdcodes2_obs=pd.read_gbq(query, dialect="standard",progress_bar_type="tqdm_notebook")
    
    icdcodes=pd.concat([icdcodes1_cond,icdcodes2_cond,icdcodes1_obs,icdcodes2_obs]).drop_duplicates() # drop duplicates within vocab, person id and date

    patients_phcode_count=icdcodes[["person_id","start_date","concept_code","vocabulary_id"]].drop_duplicates()[["person_id","concept_code","start_date","vocabulary_id"]]

    patients_phcode_count=pd.merge(phecodes[["code","phecode","vocabulary_id"]],patients_phcode_count,left_on=["code", "vocabulary_id"],right_on=["concept_code", "vocabulary_id"])

    return patients_phcode_count

In [ ]:
ICD9_IC10_Phecodes=pd.read_csv(f"{my_bucket}/data/phewas/phecode_map_icd9_10.csv")
# strip off extra column 
ICD9_IC10_Phecodes = ICD9_IC10_Phecodes.iloc[:, 1:]

In [ ]:
phecodes_list=ICD9_IC10_Phecodes["phecode"].unique().tolist()

In [ ]:
return_dict={}
phecodes_patients_list=getPhecodeParticipants(1, phecodes_list, CDR_version, 235)

In [ ]:
rollup_map=pd.read_csv('phecode_rollup_map.csv')
rollup_map

phecodes_patients_list_merge = pd.merge(rollup_map, phecodes_patients_list,left_on = 'code', right_on = 'phecode')[["person_id","phecode_unrolled","start_date"]].drop_duplicates()

phecodes_patients_counts_tmp=phecodes_patients_list_merge[["person_id","phecode_unrolled","start_date"]].groupby(["person_id","phecode_unrolled"],as_index=False).count()

phecodes_patients_counts_tmp.columns = ["person_id", "phecode", 'count']

phecodes_patients_counts_tmp.to_csv(f'{my_bucket}/data/phewas/phecodes_patient_counts.csv',index=False)

phecodes_patients_counts_tmp

In [ ]:
phecodes_patients_counts = pd.read_csv(f'{my_bucket}/data/phewas/phecodes_patient_counts.csv')

## combine everything and define phewas object

In [ ]:
# Update to sex_specific phecodes 

class PheWAS_Pool:
    """
    Class for performing PheWAS
    ======================================================================================================
    phecode_counts: Pandas Dataframe of Phecodes 
    covariates: Pandas Dataframe of covariates to include in the analysis
    indep_var: String indicating the column in covariates that is the independent variable of interest
    CDR_version: String indicating CDR version
    phecode_process: list for phecodes to process
    min_cases: minimum number of cases for an individual phenotype to be analyzed
    cores: if not "", then specify number of cores to use in the analysis 
    """
    def __init__(self, phecode_counts,
                 covariates, 
                 indep_var_of_interest="", 
                 CDR_version='R2019Q4R3',
                 phecode_process = 'all', 
                 min_cases = 100,
                 independent_var_names=["AF","white","Asian","male","age_at_last_event",
                                        "ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
                 genderspec_independent_var_names=["AF","white","Asian","age_at_last_event","unk_sex","male",
                                                   "ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
                 show_res = False,
                 cores=""):
        print("~~~~~~~~~~~~~~~        Creating PheWAS AOU Object           ~~~~~~~~~~~~~~~~~~~~~")
        # create instance attributes
        self.indep_var_of_interest = indep_var_of_interest
        #update 09_5_2019: only process phecodes passed in phecode counts
        if phecode_process =='all':
            self.phecode_list = phecode_counts["phecode"].unique().tolist()
        else:
            self.phecode_list = phecode_process
        self.CDR_version = CDR_version
        self.cores = cores 
        print("~~~~~~~~~~~~~~~       Merging Phecodes and Covariates       ~~~~~~~~~~~~~~~~~~~~~")
        self.demo_patients_phecodes = pd.merge(covariates,phecode_counts, on = ["person_id"])
        self.show_res = show_res
        self.independent_var_names = independent_var_names
        self.independent_var_names= list(np.append(np.array([self.indep_var_of_interest]),self.independent_var_names))  
        self.genderspec_independent_var_names = genderspec_independent_var_names
        self.genderspec_independent_var_names= list(np.append(np.array([self.indep_var_of_interest]),self.genderspec_independent_var_names))  
        self.remove_dup = list(np.append(np.array(["person_id"]),self.independent_var_names))
        self.min_cases = min_cases 
        
    def runPheLogit(self, phecodes): 
        #placeholder
        temp=0


In [ ]:
# Instantiate an object of class PheWAS for FULL PheWAS
test_pool = PheWAS_Pool(phecode_counts = phecodes_patients_counts,
              covariates= demo_patients_cov, CDR_version= CDR_version,indep_var_of_interest="celiac",
             independent_var_names=["AF","white","Asian","male","age_at_last_event","ehr_length","code_cnt", "unk_sex", "race_unk", "hisp_lat","unk_eth"],
            genderspec_independent_var_names=["AF","white","Asian","age_at_last_event","ehr_length","code_cnt","race_unk","hisp_lat","unk_eth"],
              phecode_process = 'all',show_res = True,
              cores =16,
        )

In [ ]:
with open('phewas_pool_v2.pkl', 'wb') as outp:
    pickle.dump(test_pool, outp, pickle.HIGHEST_PROTOCOL)